In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')

from tqdm import tqdm

import warnings
warnings.simplefilter('ignore')

import torch

import neptune.new as neptune
run = neptune.init(
    project="victorcallejas/Belluga",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJlNDRlNTJiNC00OTQwLTQxYjgtYWZiNS02OWQ0MDcwZmU5N2YifQ=="
)

from src.model.CrossVit import VisionTransformer, crossvit_base_448, crossvit_base_244

from src.data.dataloaders import pretrain_dataloader

from vit_pytorch import Dino


https://app.neptune.ai/victorcallejas/Belluga/e/BEL-92
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


 10%|█         | 596/5902 [00:22<02:50, 31.17it/s]

In [ ]:
device = torch.device("cuda")

In [ ]:
model = crossvit_base_448().to(device)

fp16 = False
input_dtype = torch.float16 if fp16 else torch.float32

In [4]:
# DINO

learner = Dino(
    model,
    image_size = 256,
    hidden_layer = 'dino_emb',         # hidden layer name or index, from which to extract the embedding
    projection_hidden_size = 256,      # projector network hidden dimension
    projection_layers = 2,             # number of layers in projection network
    num_classes_K = 65336,             # output logits dimensions (referenced as K in paper)
    student_temp = 0.9,                # student temperature
    teacher_temp = 0.04,               # teacher temperature, needs to be annealed from 0.04 to 0.07 over 30 epochs
    local_upper_crop_scale = 0.4,      # upper bound for local crop - 0.4 was recommended in the paper 
    global_lower_crop_scale = 0.5,     # lower bound for global crop - 0.5 was recommended in the paper
    moving_average_decay = 0.9,        # moving average of encoder - paper showed anywhere from 0.9 to 0.999 was ok
    center_moving_average_decay = 0.9, # moving average of teacher centers - paper showed anywhere from 0.9 to 0.999 was ok
).to(device)

opt = torch.optim.Adam(learner.parameters(), lr = 3e-4)
opt.zero_grad(True)

while True:
    for image in tqdm(pretrain_dataloader):
        image = image.to(device, non_blocking=True, dtype=input_dtype)
        loss = learner(image)
        opt.zero_grad(True)
        loss.backward()
        opt.step()
        learner.update_moving_average() # update moving average of teacher encoder and teacher centers
        run['dino/loss'].log(loss)

    

  4%|▍         | 41/984 [01:01<23:26,  1.49s/it]


KeyboardInterrupt: 

In [5]:
torch.save(model.state_dict(), 'pre.pt')